![header](../figures/logos_partenaires._cmems_se.jpg)


<div class="alert alert-block alert-success">
<h1><center>Demo: Select and save regional data </center></h1>
<h2><center>Save local netcdf from downloaded netcdf</center></h2></div>

***
**Authors:**  Datlas, IGE, IMEDEA & CLS <br>
**Copyright:** 2022 Datlas, IGE, IMEDEA & CLS <br>
**License:** MIT

In [1]:
import os
import glob
import sys
sys.path.append('..')
import numpy as np 
import zcollection
import xarray as xr 
import matplotlib.pyplot as plt
import datetime

from src.dict_regions_info import *

# From V1.0.2_CALVAL

## Reading data

In [2]:
path = '/Users/sammymetref/Documents/DataChallenges/2024b_DC_SWOTdenoising/V1.0.2_CALVAL/' 


In [3]:
%%time 

t0 = datetime.date(2023, 5, 1) 
tN = datetime.date(2023, 5, 11)

variables = ['longitude', 'latitude', 'pass_number', 'cycle_number', 'time',
             'duacs_ssha_karin_2_calibrated', 'duacs_editing_flag', 
             'duacs_ssha_karin_2_filtered', 'swh_model', 'distance_to_coast',
             'duacs_speed_zonal', 'duacs_speed_meridional', 'duacs_relative_vorticity'
            ]

zc = zcollection.open_collection(path)
zc_ds = zc.load( selected_variables=variables, 
                 filters = lambda keys: t0 <= datetime.date(keys['year'], keys['month'], keys["day"]) <= tN
) 

ds_pass = zc_ds.to_xarray().load()
ds_pass = ds_pass.set_coords(['longitude', 'latitude'])
#ds_pass = ds_pass.where(ds_pass.pass_number <= 2, drop = True)

ds_pass['ssha_karin_2_edited'] = xr.where(ds_pass.duacs_editing_flag == 0, ds_pass.duacs_ssha_karin_2_calibrated, np.nan)
ds_pass['residuals'] = ds_pass.ssha_karin_2_edited - ds_pass.duacs_ssha_karin_2_filtered

# Need to transform 0,360 longitude into -180,180
lon_attrs = ds_pass.longitude.attrs
ds_pass.coords['longitude'] = (ds_pass.coords['longitude'] + 180) % 360 - 180 
ds_pass.longitude.attrs = lon_attrs

CPU times: user 1min 3s, sys: 1min 46s, total: 2min 50s
Wall time: 4min 12s


2024-11-13 00:34:05,158 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 10 memory: 35 MB fds: 84>>
Traceback (most recent call last):
  File "/Users/sammymetref/miniconda3/envs/env-dc-swot-filtering/lib/python3.10/site-packages/tornado/ioloop.py", line 937, in _run
    val = self.callback()
  File "/Users/sammymetref/miniconda3/envs/env-dc-swot-filtering/lib/python3.10/site-packages/distributed/system_monitor.py", line 160, in update
    net_ioc = psutil.net_io_counters()
  File "/Users/sammymetref/miniconda3/envs/env-dc-swot-filtering/lib/python3.10/site-packages/psutil/__init__.py", line 2117, in net_io_counters
    rawdict = _psplatform.net_io_counters()
OSError: [Errno 12] Cannot allocate memory


In [4]:

path_data = '../swot_data/V1.0.2_CALVAL/SWOT_L3_LR_SSH_Expert'

## Selecting and saving Gulf Stream region

In [36]:
%%time
 
region_info = RegionInfo('GS_info').region_info   # Region dictionary info from dict_regions_info.py
 

ds_pass1 = ds_pass.where(ds_pass['longitude'].compute()<region_info['lon_max'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['longitude'].compute()>region_info['lon_min'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['latitude'].compute()<region_info['lat_max'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['latitude'].compute()>region_info['lat_min'],drop=True)
ds_pass1.time.attrs['_FillValue'] = np.nan

cycle_number = np.nanmax(ds_pass1.cycle_number,axis=1) 
ds_pass1 = ds_pass1.drop_vars({'cycle_number'})
ds_pass1['cycle_number'] = (['num_lines', ], cycle_number)

pass_number = np.nanmax(ds_pass1.pass_number,axis=1) 
ds_pass1 = ds_pass1.drop_vars({'pass_number'})
ds_pass1['pass_number'] = (['num_lines', ], pass_number)


ds_pass1.to_netcdf(path_data+'_'+region_info['name']+'_v1.0.2.nc')

#del ds_pass1

CPU times: user 59.1 s, sys: 2min 15s, total: 3min 14s
Wall time: 8min 19s


## Selecting and saving Mediterranean region

In [38]:
%%time 

region_info = RegionInfo('Med_info').region_info   # Region dictionary info from dict_regions_info.py

ds_pass1 = ds_pass.where(ds_pass['longitude'].compute()<region_info['lon_max'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['longitude'].compute()>region_info['lon_min'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['latitude'].compute()<region_info['lat_max'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['latitude'].compute()>region_info['lat_min'],drop=True)
ds_pass1.time.attrs['_FillValue'] = np.nan

cycle_number = np.nanmax(ds_pass1.cycle_number,axis=1) 
ds_pass1 = ds_pass1.drop_vars({'cycle_number'})
ds_pass1['cycle_number'] = (['num_lines', ], cycle_number)

pass_number = np.nanmax(ds_pass1.pass_number,axis=1) 
ds_pass1 = ds_pass1.drop_vars({'pass_number'})
ds_pass1['pass_number'] = (['num_lines', ], pass_number)

ds_pass1.to_netcdf(path_data+'_'+region_info['name']+'_v1.0.2.nc')

del ds_pass1

<timed exec>:9: RuntimeWarning: All-NaN axis encountered
<timed exec>:13: RuntimeWarning: All-NaN axis encountered


CPU times: user 1min 9s, sys: 2min 44s, total: 3min 54s
Wall time: 9min 12s


## Selecting and saving Antarctic Circumpolar region

In [39]:
%%time 

region_info = RegionInfo('ACC_info').region_info   # Region dictionary info from dict_regions_info.py

ds_pass1 = ds_pass.where(ds_pass['longitude'].compute()<region_info['lon_max'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['longitude'].compute()>region_info['lon_min'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['latitude'].compute()<region_info['lat_max'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['latitude'].compute()>region_info['lat_min'],drop=True)
ds_pass1.time.attrs['_FillValue'] = np.nan

cycle_number = np.nanmax(ds_pass1.cycle_number,axis=1) 
ds_pass1 = ds_pass1.drop_vars({'cycle_number'})
ds_pass1['cycle_number'] = (['num_lines', ], cycle_number)

pass_number = np.nanmax(ds_pass1.pass_number,axis=1) 
ds_pass1 = ds_pass1.drop_vars({'pass_number'})
ds_pass1['pass_number'] = (['num_lines', ], pass_number)

ds_pass1.to_netcdf(path_data+'_'+region_info['name']+'_v1.0.2.nc')

del ds_pass1

<timed exec>:9: RuntimeWarning: All-NaN axis encountered
<timed exec>:13: RuntimeWarning: All-NaN axis encountered


CPU times: user 1min 39s, sys: 3min 52s, total: 5min 32s
Wall time: 11min 56s


## Selecting and saving Madagascar region

In [40]:
%%time
 
region_info = RegionInfo('Mada_info').region_info   # Region dictionary info from dict_regions_info.py

ds_pass1 = ds_pass.where(ds_pass['longitude'].compute()<region_info['lon_max'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['longitude'].compute()>region_info['lon_min'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['latitude'].compute()<region_info['lat_max'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['latitude'].compute()>region_info['lat_min'],drop=True)
ds_pass1.time.attrs['_FillValue'] = np.nan

cycle_number = np.nanmax(ds_pass1.cycle_number,axis=1) 
ds_pass1 = ds_pass1.drop_vars({'cycle_number'})
ds_pass1['cycle_number'] = (['num_lines', ], cycle_number)

pass_number = np.nanmax(ds_pass1.pass_number,axis=1) 
ds_pass1 = ds_pass1.drop_vars({'pass_number'})
ds_pass1['pass_number'] = (['num_lines', ], pass_number)

ds_pass1.to_netcdf(path_data+'_'+region_info['name']+'_v1.0.2.nc')

del ds_pass1

CPU times: user 1min 9s, sys: 2min 52s, total: 4min 2s
Wall time: 7min 23s


## Selecting and saving Californian region

In [41]:
%%time

region_info = Cali_info

ds_pass1 = ds_pass.where(ds_pass['longitude'].compute()<region_info['lon_max'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['longitude'].compute()>region_info['lon_min'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['latitude'].compute()<region_info['lat_max'],drop=True)
ds_pass1 = ds_pass1.where(ds_pass1['latitude'].compute()>region_info['lat_min'],drop=True)
ds_pass1.time.attrs['_FillValue'] = np.nan

cycle_number = np.nanmax(ds_pass1.cycle_number,axis=1) 
ds_pass1 = ds_pass1.drop_vars({'cycle_number'})
ds_pass1['cycle_number'] = (['num_lines', ], cycle_number)

pass_number = np.nanmax(ds_pass1.pass_number,axis=1) 
ds_pass1 = ds_pass1.drop_vars({'pass_number'})
ds_pass1['pass_number'] = (['num_lines', ], pass_number)

ds_pass1.to_netcdf(path_data+'_'+region_info['name']+'_v1.0.2.nc')

del ds_pass1

CPU times: user 33.2 s, sys: 1min 22s, total: 1min 55s
Wall time: 3min 33s
